In [1]:
import logging
logging.basicConfig(
  level=logging.INFO,  
  format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",  
)
logging.getLogger("httpx").setLevel(logging.WARNING)
LOGGER = logging.getLogger(__name__)

import os
import pandas as pd
from typing_extensions import override
from bondable.bond.config import Config
from bondable.bond.functions import Functions, DefaultFunctions
from bondable.bond.agent import Agent
from bondable.bond.builder import AgentBuilder, AgentDefinition
from IPython.display import Image, display
from dotenv import load_dotenv


load_dotenv(override=True)

user_id = "test@user.com"

config = Config.config()
builtin_functions = Functions.functions()
builder = AgentBuilder.builder()
builder.cleanup()

2025-03-10 15:31:11,005 - bondable.bond.config - INFO - Created Config instance
2025-03-10 15:31:11,005 - bondable.bond.functions - INFO - Created Functions instance using class: (bondable.bond.functions.DefaultFunctions)
2025-03-10 15:31:11,011 - bondable.bond.builder - INFO - Created AgentBuilder instance using database engine: sqlite:///.bondcache.db


In [4]:
agent_def = AgentDefinition(
    name="Simple Agent",
    description="Pirate Agent.",
    instructions="Answer requests from user like a pirate.",
    metadata={'visible': 'True'},
    tools=[{"type": "code_interpreter"}]
)
# simple_agent = builder.get_agent(agent_def)
# print(simple_agent)
builder.print_responses(user_id, ['tell me a joke'], agent_def)

2025-03-10 15:32:18,746 - bondable.bond.builder - INFO - Creating Agent Definition with tool -> {'type': 'code_interpreter'}
2025-03-10 15:32:19,424 - bondable.bond.builder - INFO - Created new agent [Simple Agent] with assistant_id: asst_Sk4Q6wohAkMECN8F3Dd5MyhV


[msg_jsJDtqG7pXWYwKjUyxtCnAHj/user] => tell me a joke
[msg_6B0kPpsVOj4OpAP5d7XG8qb7_0/assistant] => Arrr, matey! Why did the pirate go to school?

Because he wanted to improve his "arrrticulation!" Har har har! 🏴‍☠️


In [5]:
agent_def = AgentDefinition(
    name="Function Agent",
    description="Say hello to the user.",
    instructions="Call the simple method 'hello' with the name provided by the user. If no name is provided, ask the user to provide a name.",
    metadata={'visible': 'True'},
    tools=[builtin_functions.hello],
)
builder.print_responses(user_id, ['my name is john'], agent_def)

2025-03-10 15:32:30,978 - bondable.bond.builder - INFO - Creating Agent Definition with tool -> {'type': 'function', 'function': {'name': 'hello', 'description': 'Will say hello to the name provided.', 'strict': False, 'parameters': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name to say hello to.'}}, 'required': ['name']}}}
2025-03-10 15:32:31,647 - bondable.bond.builder - INFO - Created new agent [Function Agent] with assistant_id: asst_KlbpwHnhFmoYlLcRnXWW0iCf


[msg_LXGhXL1Rx5qUZ2pvdFSN0JIt/user] => my name is john


2025-03-10 15:32:34,768 - bondable.bond.functions - INFO - Saying hello to: john


[msg_GVtuRmnMNAZf4O5KblcEvC3v_0/assistant] => Hello, john!


In [ ]:
agent_def = AgentDefinition(
    name="Synth Agent",
    description="An agent that will load synthetic data about people and answer questions",
    instructions="""When you begin you should create a synthetic data set that has the weight 
    and height of 100 people. You should answer questions about the data set
    """,
    tools=[{"type": "code_interpreter"}],
    metadata={'visible': 'True', 'initial_prompt': 'Generate the data set'},

)
agent = builder.get_agent(agent_def)
prompts = [
  agent.get_metadata_value('initial_prompt'),
  "How many people are there",
  "What is the average height",
  "Show me a scatterplot of height vs weight"
]
builder.print_responses(user_id, prompts, agent_def)



In [3]:
# Create a sample data of users
data = {
  'Customer_ID': [1, 2, 3],
  'First_Name': ['Jack', 'Jane', 'Doe'],
  'Last_Name': ['Doe', 'Smith', 'Johnson'],
  'Email': ['john.doe@example.com', 'jane.smith@example.com', 'doe.johnson@example.com'],
  'Phone_Number': ['123-456-7890', '234-567-8901', '345-678-9012'],
  'Region': ['North', 'South', 'East'],
  'Salesperson': ['Alice', 'Bob', 'Charlie'],
  'Address': ['123 Elm St', '456 Oak St', '789 Pine St'],
  'ZIP_Code': ['12345', '23456', '34567'],
  'Last_Purchase_Amount': [100.50, 200.75, 150.00]
}
df = pd.DataFrame(data)
data_file = 'sample_customers.csv'
df.to_csv(data_file, index=False)

agent_def = AgentDefinition(
    name="File Agent",
    description="An agent that will load data about people and answer questions",
    instructions="""
    Answer questions about user data in the attached CSV file. The first row contains the
    names of the columns. 
    """,
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "files": [data_file]
      }
    },
    metadata={'visible': 'True', 'initial_prompt': 'What are all of the customer names?'},
)
agent = builder.get_agent(agent_def)
prompts = [
  agent.get_metadata_value('initial_prompt'),
]
builder.print_responses(user_id, prompts, agent_def)




2025-03-10 15:31:51,466 - bondable.bond.builder - INFO - Creating Agent Definition with tool -> {'type': 'code_interpreter'}
2025-03-10 15:31:51,468 - bondable.bond.builder - INFO - Created new file record for sample_customers.csv
2025-03-10 15:31:53,675 - bondable.bond.builder - INFO - Created new agent [File Agent] with assistant_id: asst_i4EFohIUN9eeGQPkXb2WcoER
2025-03-10 15:31:55,169 - bondable.bond.builder - INFO - Creating Agent Definition with tool -> {'type': 'code_interpreter'}
2025-03-10 15:31:55,170 - bondable.bond.builder - INFO - Reusing agent [File Agent] with assistant_id: asst_i4EFohIUN9eeGQPkXb2WcoER


[msg_ffbwdpudOup9HHYOtGeY0UOB/user] => What are all of the customer names?
[msg_rFnCKfNa4P79Z23DQ54sEgGd_0/assistant] => The customer names are:

1. Jack Doe
2. Jane Smith
3. Doe Johnson


In [2]:
# Create a sample data of users
data = {
  'Customer_ID': [1, 2, 3],
  'First_Name': ['Jack', 'Jane', 'Doe'],
  'Last_Name': ['Doe', 'Smith', 'Johnson'],
  'Email': ['john.doe@example.com', 'jane.smith@example.com', 'doe.johnson@example.com'],
  'Phone_Number': ['123-456-7890', '234-567-8901', '345-678-9012'],
  'Region': ['North', 'South', 'East'],
  'Salesperson': ['Alice', 'Bob', 'Charlie'],
  'Address': ['123 Elm St', '456 Oak St', '789 Pine St'],
  'ZIP_Code': ['12345', '23456', '34567'],
  'Last_Purchase_Amount': [100.50, 200.75, 150.00]
}
df = pd.DataFrame(data)
html_file = 'sample_customers.html'
df.to_html(html_file, index=False)

agent_def = AgentDefinition(
    name="HTML File Agent",
    description="An agent that will load data about people and answer questions",
    instructions="""
    Answer questions about user data in the attached HTML file.
    """,
    tools=[{"type": "file_search"}],
    tool_resources={
      "file_search": {
        "files": [html_file]
      }
    },
    metadata={'visible': 'True', 'initial_prompt': 'What are all of the customer names?'},
)
agent = builder.get_agent(agent_def)
prompts = [
  agent.get_metadata_value('initial_prompt'),
]
builder.print_responses(user_id, prompts, agent_def)

2025-03-10 15:31:20,383 - bondable.bond.builder - INFO - Creating Agent Definition with tool -> {'type': 'file_search'}
2025-03-10 15:31:20,724 - bondable.bond.builder - INFO - Created new vector store HTML File Agent_file_search with vector_store_id: vs_67cf6838aa4c81919544bd79c4a027aa
2025-03-10 15:31:20,966 - bondable.bond.builder - INFO - Created new file record for sample_customers.html
2025-03-10 15:31:23,754 - bondable.bond.builder - INFO - Created new vector store [vs_67cf6838aa4c81919544bd79c4a027aa] file record for file: sample_customers.html
2025-03-10 15:31:25,046 - bondable.bond.broker - INFO - Created Broker instance
2025-03-10 15:31:25,048 - bondable.bond.builder - INFO - Created new agent [HTML File Agent] with assistant_id: asst_RgmmUNropmpNUMAhSNp6rqEb
2025-03-10 15:31:25,882 - bondable.bond.builder - INFO - Creating Agent Definition with tool -> {'type': 'file_search'}
2025-03-10 15:31:25,883 - bondable.bond.builder - INFO - Reusing agent [HTML File Agent] with assis

[msg_0unEtLiRzco4KyS3XFZIfJI9/user] => What are all of the customer names?
[msg_dA1AoiNESkbo6O5D6SfMKCiC_0/assistant] => The customer names listed in the data are as follows:

1. Jack Doe
2. Jane Smith
3. Doe Johnson

These names can be found in the provided HTML file【4:0†source】.


In [ ]:
# create an agent that uses a tool written as a python method to answer questions
# the python method 'execute_sql_googlesheets' lives in fumctions.py
# this function will execute a SQL query on a google sheet

googlesheet_assistant = openai_client.beta.assistants.update(
    assistant_id=existing_assistants["Googlesheet Agent"],
    name="Googlesheet Agent",
    instructions="""
    You are an expert answering questions about customers. When asked a question, you should
    translate the question into a SQL query and execute it using the function execute_sql_googlesheets.
    The table name is 'df' and the columns are:
    - Customer_ID
    - First_Name
    - Last_Name
    - Email
    - Phone_Number
    - Region
    - Salesperson
    - Address
    - ZIP_Code
    - Last_Purchase_Amount

    An example question might be "How many customers are in each regio" which you would translate into:
    SELECT Region, COUNT(*) FROM df GROUP BY Region
    """,
    tools=[{"type": "code_interpreter"},
       {
        "type": "function",
        "function": {
          "name": "execute_sql_googlesheets",
          "description": "execute a sql query on a google sheet",
          "parameters": {
            "type": "object",
            "properties": {
              "sql_query": {
                "type": "string",
                "description": "the query to execute"
              }
            },
            "required": ["sql_query"]
          }
        }
      }],
    model=openai_deployment,
)

googlesheet_agent = Agent(assistant_id=googlesheet_assistant.id, functions=builtin_functions)


In [ ]:

thread = openai_client.beta.threads.create()
ask_agent_stream("how many customers are there", thread, googlesheet_agent)
response = openai_client.beta.threads.delete(thread.id)

In [ ]:
# create an agent that uses two tools written as python methods to answer requests
# both python methods lives in fumctions.py
# the function 'execute_sql_googlesheets' will execute a SQL query on a google sheet
# the function 'send_email' will send an email

message_assistant = openai_client.beta.assistants.update(
    assistant_id=existing_assistants["Message Action Agent"],
    name="Message Action Agent",
    instructions="""
    You are an expert answering questions about customers. When asked a question, you should
    translate the question into a SQL query and execute it using the function execute_sql_googlesheets.
    The table name is 'df' and the columns are:
    - Customer_ID
    - First_Name
    - Last_Name
    - Email
    - Phone_Number
    - Region
    - Salesperson
    - Address
    - ZIP_Code
    - Last_Purchase_Amount

    An example question might be "How many customers are in each regio" which you would translate into:
    SELECT Region, COUNT(*) FROM df GROUP BY Region

    You can also send a message with results for any user questions using the function 'send_email'.
    THis function takes a message body, a subject and a to email address as parameters. 
    """,
    tools=[{"type": "code_interpreter"},
       {
        "type": "function",
        "function": {
          "name": "execute_sql_googlesheets",
          "description": "execute a sql query on a google sheet",
          "parameters": {
            "type": "object",
            "properties": {
              "sql_query": {
                "type": "string",
                "description": "the query to execute"
              }
            },
            "required": ["sql_query"]
          }
        }
      },
      {
        "type": "function",
        "function": {
          "name": "send_email",
          "description": "send an email",
          "parameters": {
            "type": "object",
            "properties": {
              "to": {
                "type": "string",
                "description": "the to address"
              },
              "subject": {
                "type": "string",
                "description": "the subject of the email"
              },
              "body": {
                "type": "string",
                "description": "the body of the email"
              }
            },
            "required": ["to", "subject", "body"]
          }
        }
      },
      ],
    model=openai_deployment,
)

In [ ]:
agent = Agent(assistant_id=message_assistant.id, functions=builtin_functions)
thread = openai_client.beta.threads.create()
ask_agent("how many customers are there", thread, agent)
response = openai_client.beta.threads.delete(thread.id)